# DF

In [ ]:
import marimo as mo
import polars as pl

from r_translation import join

df = join()

# matching and contentious split
What makes a data point contentious is where it has duplicate speciesId.

In [ ]:
matching = df.filter(~pl.col("speciesId").is_duplicated()).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(-1))
    .cast(pl.Int64)
)
contentious = df.filter((pl.col("speciesId").is_duplicated())).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(-1))
    .cast(pl.Int64)
)

unique_contentious = contentious.filter(
    (pl.col("acceptedNameUsageID") == -1)
    & (
        pl.col("matched_taxonID").is_in(
            pl.col("acceptedNameUsageID").implode()
        )
    )
)
# print(
#     unique_contentious.select(
#         "speciesId", "acceptedNameUsageID", "matched_taxonID"
#     ).collect()
# )
unique_contentius_speciesId = (
    unique_contentious.select("speciesId").collect().to_series().implode()
)  # Just the speciesIds
contentious2 = contentious.filter(
    ~pl.col("speciesId").is_in(unique_contentius_speciesId)
)  # Removing...

# print(contentious2.select('speciesId','acceptedNameUsageID','matched_taxonID').collect())

matching = pl.concat(
    [matching, unique_contentious],
)

In [ ]:
contentious2.collect().sort("speciesId")

In [ ]:
matching.collect().sort("speciesId")

nomatch lazyframe is for BOS data point that has no current match in gbif data set.

In [ ]:
nomatch = (
    matching.filter(pl.col("matched_taxonID").is_null())
    .with_columns(
        taxonRank=pl.lit("BOSuncornirmedSpecies"),
        taxonomicStatus=pl.lit("BOSunformired"),
        parentNameUsageID=pl.lit(None),
    )
    .collect()
)
nomatch

speciesId,matched_taxonID,acceptedNameUsageID,taxonName,domain,kingdom,phylum,class,subclass,superorder,order,sub-order,infraorder,section,subsection,superfamily,family,subfamily,tribe,genus,subgenus,genericName,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,parentNameUsageID
str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null
"""A-Anne-Clit-Hirudinea-000001""",null,-1,"""Glossiphoniidae sp. ""1""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Rhynchobdellida""",null,null,null,null,null,"""Glossiphoniidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000002""",null,-1,"""Glossiphoniidae sp. ""2""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Rhynchobdellida""",null,null,null,null,null,"""Glossiphoniidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000003""",null,-1,"""Glossiphoniidae sp. ""3""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Rhynchobdellida""",null,null,null,null,null,"""Glossiphoniidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000004""",null,-1,"""Hirudinidae sp. ""1""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Arhynchobdellida""",null,null,null,null,null,"""Hirudinidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000005""",null,-1,"""Hirudinidae sp. ""2""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Arhynchobdellida""",null,null,null,null,null,"""Hirudinidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""A-Hemi-Enteropneusta-000010""",null,-1,"""Acorn worm sp.""",null,null,"""Hemichordata""","""Enteropneusta""",null,null,"""blank-UnID""",null,null,null,null,null,null,null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Hemi-Enteropneusta-000011""",null,-1,"""Acorn worm sp.""",null,null,"""Hemichordata""","""Enteropneusta""",null,null,"""blank-UnID""",null,null,null,null,null,null,null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Phoronida-000002""",null,-1,"""Phoronida sp.""",null,null,"""Phoronida""","""blank-Phor""",null,null,"""blank-Phor""",null,null,null,null,null,"""Phoronidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null


In [ ]:
accepted_taxon_lookup = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(pl.col("taxonomicStatus") == pl.lit("accepted"))
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .filter(~pl.col("canonicalName").is_null())
    .filter(pl.col("canonicalName") != "")
    .sort("canonicalName")
)

In [ ]:
# accepted_taxon_lookup.filter(
#     pl.col("canonicalName") == "Abakaniella"
# ).collect()

In [ ]:
# unique_accepted_taxons = accepted_taxon_lookup.filter(
#     ~pl.col("canonicalName").is_duplicated()
# )
# unique_accepted_taxons.group_by("canonicalName").len().sort("len").collect()

In [ ]:
repeated_accepted_taxons = accepted_taxon_lookup.filter(
    pl.col("canonicalName").is_duplicated()
)
# repeated_accepted_taxons.group_by("canonicalName").len().sort("len").collect()

### repeated_accepted_taxons = RAT

In [ ]:
# repeated_accepted_taxons.collect_schema().keys()

In [ ]:
priority_columns = [
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
    # "domain",
]

_schema = {
    "feature_that_is_equal_to_canonicalName": pl.String,
    "matches": pl.String,
}

RAT_interim = pl.DataFrame(schema=_schema)
for _c in priority_columns:
    _a = (
        repeated_accepted_taxons.filter(pl.col("canonicalName") == pl.col(_c))
        .select("canonicalName")
        .unique()
        .collect()
    )
    if _a.shape[0] != 0:
        _t = _a["canonicalName"].to_list()

        _row = pl.DataFrame(
            data={"feature_that_is_equal_to_canonicalName": _c, "matches": _t},
            schema=_schema,
        )
        RAT_interim = RAT_interim.vstack(_row)
RAT_feats = (
    RAT_interim.group_by("matches")
    .agg(pl.col("feature_that_is_equal_to_canonicalName").str.join(", "))
    .with_columns(
        pl.col("feature_that_is_equal_to_canonicalName").str.split(", ")
    )
    .sort("feature_that_is_equal_to_canonicalName")
)
RAT_feats

matches,feature_that_is_equal_to_canonicalName
str,list[str]
"""Micrognathozoa""","[""class"", ""phylum""]"
"""Sirenia""","[""family"", ""order""]"
"""Passeriformes""","[""family"", ""order""]"
"""Eurypetalum""","[""genus""]"
"""Dugesia""","[""genus""]"
…,…
"""Diplura""","[""genus"", ""order"", ""class""]"
"""Ctenophora""","[""genus"", ""phylum""]"
"""Acanthocephala""","[""genus"", ""phylum""]"


In [ ]:
accepted_taxon_lookup.columns

/tmp/marimo_210861/__marimo__cell_ZHCJ_.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  accepted_taxon_lookup.columns


In [ ]:
nomatch.columns

In [ ]:
priorityFeatures = [
    # "canonicalName",
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
]
pl.Config.set_tbl_cols(1000)


# df4 = pl.DataFrame()
for _f in priorityFeatures:
    for _m in RAT_feats["matches"]:
        _nomatch_subset = nomatch.filter(pl.col(_f) == _m)
        if _nomatch_subset.shape[0] == 0:
            continue

        # _nomatch_subset.with_columns(parentNameUsageID = pl.col('taxon_id')) # THis is not correct

        print("-------Feature", _f, "name", _m)
        # print("nomatch.filter(pl.col(_feature) == _name)")
        # print(
        #     _nomatch_subset.select(
        #         [
        #             # "speciesId",
        #             # "acceptedNameUsageID",
        #             "taxonName",
        #             # "taxonRank",
        #             # "taxonomicStatus",
        #         ]
        #         + priorityFeatures
        #     )
        # )
        print(
            "repeated_accepted_taxons.filter(pl.col('canonicalName') == _name)"
        )
        print(
            repeated_accepted_taxons.filter(pl.col("canonicalName") == _m)
            .select(["taxonID"] + priorityFeatures)
            .collect()
        )

        print("\n\n\n\n")
        # df4 = df4.vstack(_nomatch_subset)

-------Feature family name Sciaridae
nomatch.filter(pl.col(_f) == _m)
shape: (111, 9)
┌────────────┬───────────┬───────────┬───────┬───────────┬─────────┬─────────┬───────────┬─────────┐
│ taxonName  ┆ infraspec ┆ specificE ┆ genus ┆ family    ┆ order   ┆ class   ┆ phylum    ┆ kingdom │
│ ---        ┆ ificEpith ┆ pithet    ┆ ---   ┆ ---       ┆ ---     ┆ ---     ┆ ---       ┆ ---     │
│ str        ┆ et        ┆ ---       ┆ str   ┆ str       ┆ str     ┆ str     ┆ str       ┆ str     │
│            ┆ ---       ┆ str       ┆       ┆           ┆         ┆         ┆           ┆         │
│            ┆ str       ┆           ┆       ┆           ┆         ┆         ┆           ┆         │
╞════════════╪═══════════╪═══════════╪═══════╪═══════════╪═════════╪═════════╪═══════════╪═════════╡
│ Sciaridae  ┆           ┆ null      ┆ null  ┆ Sciaridae ┆ Diptera ┆ Insecta ┆ Arthopoda ┆ null    │
│ sp. ZRC_BD ┆           ┆           ┆       ┆           ┆         ┆         ┆           ┆         │
│ P00

shape: (2, 9)
┌──────────┬───────────┬───────────┬──────────┬──────────┬─────────┬─────────┬──────────┬──────────┐
│ taxonID  ┆ infraspec ┆ specificE ┆ genus    ┆ family   ┆ order   ┆ class   ┆ phylum   ┆ kingdom  │
│ ---      ┆ ificEpith ┆ pithet    ┆ ---      ┆ ---      ┆ ---     ┆ ---     ┆ ---      ┆ ---      │
│ i64      ┆ et        ┆ ---       ┆ str      ┆ str      ┆ str     ┆ str     ┆ str      ┆ str      │
│          ┆ ---       ┆ str       ┆          ┆          ┆         ┆         ┆          ┆          │
│          ┆ str       ┆           ┆          ┆          ┆         ┆         ┆          ┆          │
╞══════════╪═══════════╪═══════════╪══════════╪══════════╪═════════╪═════════╪══════════╪══════════╡
│ 3525     ┆ null      ┆ null      ┆ null     ┆ Sciarida ┆ Diptera ┆ Insecta ┆ Arthropo ┆ Animalia │
│          ┆           ┆           ┆          ┆ e        ┆         ┆         ┆ da       ┆          │
│ 11496569 ┆ null      ┆ null      ┆ Sciarida ┆ Sciarida ┆ Diptera ┆ Insecta 

shape: (2, 9)
┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ taxonID  ┆ infraspe ┆ specific ┆ genus    ┆ family   ┆ order    ┆ class    ┆ phylum   ┆ kingdom  │
│ ---      ┆ cificEpi ┆ Epithet  ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ i64      ┆ thet     ┆ ---      ┆ str      ┆ str      ┆ str      ┆ str      ┆ str      ┆ str      │
│          ┆ ---      ┆ str      ┆          ┆          ┆          ┆          ┆          ┆          │
│          ┆ str      ┆          ┆          ┆          ┆          ┆          ┆          ┆          │
╞══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 10768934 ┆ null     ┆ null     ┆ Pilargid ┆ Pilargid ┆ Phyllodo ┆ Polychae ┆ Annelida ┆ Animalia │
│          ┆          ┆          ┆ ae       ┆ ae       ┆ cida     ┆ ta       ┆          ┆          │
│ 7077     ┆ null     ┆ null     ┆ null     ┆ Pilargid ┆ Phyllodo ┆ Polychae 

shape: (2, 9)
┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ taxonID  ┆ infraspe ┆ specific ┆ genus    ┆ family   ┆ order    ┆ class    ┆ phylum   ┆ kingdom  │
│ ---      ┆ cificEpi ┆ Epithet  ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ i64      ┆ thet     ┆ ---      ┆ str      ┆ str      ┆ str      ┆ str      ┆ str      ┆ str      │
│          ┆ ---      ┆ str      ┆          ┆          ┆          ┆          ┆          ┆          │
│          ┆ str      ┆          ┆          ┆          ┆          ┆          ┆          ┆          │
╞══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 10897709 ┆ null     ┆ null     ┆ Hesionid ┆ Hesionid ┆ Phyllodo ┆ Polychae ┆ Annelida ┆ Animalia │
│          ┆          ┆          ┆ ae       ┆ ae       ┆ cida     ┆ ta       ┆          ┆          │
│ 6969     ┆ null     ┆ null     ┆ null     ┆ Hesionid ┆ Phyllodo ┆ Polychae 

-------Feature family name Cirratulidae
nomatch.filter(pl.col(_f) == _m)
shape: (5, 9)
┌───────────┬───────────┬───────────┬───────┬───────────┬──────────┬──────────┬──────────┬─────────┐
│ taxonName ┆ infraspec ┆ specificE ┆ genus ┆ family    ┆ order    ┆ class    ┆ phylum   ┆ kingdom │
│ ---       ┆ ificEpith ┆ pithet    ┆ ---   ┆ ---       ┆ ---      ┆ ---      ┆ ---      ┆ ---     │
│ str       ┆ et        ┆ ---       ┆ str   ┆ str       ┆ str      ┆ str      ┆ str      ┆ str     │
│           ┆ ---       ┆ str       ┆       ┆           ┆          ┆          ┆          ┆         │
│           ┆ str       ┆           ┆       ┆           ┆          ┆          ┆          ┆         │
╞═══════════╪═══════════╪═══════════╪═══════╪═══════════╪══════════╪══════════╪══════════╪═════════╡
│ Cirratuli ┆           ┆ null      ┆ null  ┆ Cirratuli ┆ Terebell ┆ Polychae ┆ Annelida ┆ null    │
│ dae sp. 1 ┆           ┆           ┆       ┆ dae       ┆ ida      ┆ ta       ┆          ┆         │
│ Ci

...

shape: (2, 9)
┌─────────┬───────────┬───────────┬──────────┬──────────┬─────────┬──────────┬──────────┬──────────┐
│ taxonID ┆ infraspec ┆ specificE ┆ genus    ┆ family   ┆ order   ┆ class    ┆ phylum   ┆ kingdom  │
│ ---     ┆ ificEpith ┆ pithet    ┆ ---      ┆ ---      ┆ ---     ┆ ---      ┆ ---      ┆ ---      │
│ i64     ┆ et        ┆ ---       ┆ str      ┆ str      ┆ str     ┆ str      ┆ str      ┆ str      │
│         ┆ ---       ┆ str       ┆          ┆          ┆         ┆          ┆          ┆          │
│         ┆ str       ┆           ┆          ┆          ┆         ┆          ┆          ┆          │
╞═════════╪═══════════╪═══════════╪══════════╪══════════╪═════════╪══════════╪══════════╪══════════╡
│ 256     ┆ null      ┆ null      ┆ null     ┆ null     ┆ null    ┆ Polychae ┆ Annelida ┆ Animalia │
│         ┆           ┆           ┆          ┆          ┆         ┆ ta       ┆          ┆          │
│ 1464857 ┆ null      ┆ null      ┆ Polychae ┆ Tachinid ┆ Diptera ┆ Insecta  

-------Feature phylum name Ctenophora
nomatch.filter(pl.col(_f) == _m)
shape: (11, 9)
┌───────────┬───────────┬───────────┬───────┬───────────┬──────────┬──────────┬──────────┬─────────┐
│ taxonName ┆ infraspec ┆ specificE ┆ genus ┆ family    ┆ order    ┆ class    ┆ phylum   ┆ kingdom │
│ ---       ┆ ificEpith ┆ pithet    ┆ ---   ┆ ---       ┆ ---      ┆ ---      ┆ ---      ┆ ---     │
│ str       ┆ et        ┆ ---       ┆ str   ┆ str       ┆ str      ┆ str      ┆ str      ┆ str     │
│           ┆ ---       ┆ str       ┆       ┆           ┆          ┆          ┆          ┆         │
│           ┆ str       ┆           ┆       ┆           ┆          ┆          ┆          ┆         │
╞═══════════╪═══════════╪═══════════╪═══════╪═══════════╪══════════╪══════════╪══════════╪═════════╡
│ <i>Coelop ┆           ┆ null      ┆ null  ┆ Coeloplan ┆ Platycte ┆ Tentacul ┆ Ctenopho ┆ null    │
│ lana</i>  ┆           ┆           ┆       ┆ idae      ┆ nida     ┆ ata      ┆ ra       ┆         │
│ sp.

shape: (2, 9)
┌─────────┬───────────┬───────────┬───────────┬──────────┬─────────┬─────────┬──────────┬──────────┐
│ taxonID ┆ infraspec ┆ specificE ┆ genus     ┆ family   ┆ order   ┆ class   ┆ phylum   ┆ kingdom  │
│ ---     ┆ ificEpith ┆ pithet    ┆ ---       ┆ ---      ┆ ---     ┆ ---     ┆ ---      ┆ ---      │
│ i64     ┆ et        ┆ ---       ┆ str       ┆ str      ┆ str     ┆ str     ┆ str      ┆ str      │
│         ┆ ---       ┆ str       ┆           ┆          ┆         ┆         ┆          ┆          │
│         ┆ str       ┆           ┆           ┆          ┆         ┆         ┆          ┆          │
╞═════════╪═══════════╪═══════════╪═══════════╪══════════╪═════════╪═════════╪══════════╪══════════╡
│ 51      ┆ null      ┆ null      ┆ null      ┆ null     ┆ null    ┆ null    ┆ Ctenopho ┆ Animalia │
│         ┆           ┆           ┆           ┆          ┆         ┆         ┆ ra       ┆          │
│ 1507726 ┆ null      ┆ null      ┆ Ctenophor ┆ Tipulida ┆ Diptera ┆ Insecta 

-------Feature phylum name Arthropoda
nomatch.filter(pl.col(_f) == _m)
shape: (2_266, 9)
┌───────────┬───────────┬───────────┬───────┬───────────┬──────────┬──────────┬──────────┬─────────┐
│ taxonName ┆ infraspec ┆ specificE ┆ genus ┆ family    ┆ order    ┆ class    ┆ phylum   ┆ kingdom │
│ ---       ┆ ificEpith ┆ pithet    ┆ ---   ┆ ---       ┆ ---      ┆ ---      ┆ ---      ┆ ---     │
│ str       ┆ et        ┆ ---       ┆ str   ┆ str       ┆ str      ┆ str      ┆ str      ┆ str     │
│           ┆ ---       ┆ str       ┆       ┆           ┆          ┆          ┆          ┆         │
│           ┆ str       ┆           ┆       ┆           ┆          ┆          ┆          ┆         │
╞═══════════╪═══════════╪═══════════╪═══════╪═══════════╪══════════╪══════════╪══════════╪═════════╡
│ Amblypygi ┆           ┆ null      ┆ null  ┆ null      ┆ Amblypyg ┆ Arachnid ┆ Arthropo ┆ null    │
│ sp. ZRC_E ┆           ┆           ┆       ┆           ┆ i        ┆ a        ┆ da       ┆         │
│ 

shape: (2, 9)
┌──────────┬────────────┬────────────┬────────────┬────────┬───────┬───────┬────────────┬──────────┐
│ taxonID  ┆ infraspeci ┆ specificEp ┆ genus      ┆ family ┆ order ┆ class ┆ phylum     ┆ kingdom  │
│ ---      ┆ ficEpithet ┆ ithet      ┆ ---        ┆ ---    ┆ ---   ┆ ---   ┆ ---        ┆ ---      │
│ i64      ┆ ---        ┆ ---        ┆ str        ┆ str    ┆ str   ┆ str   ┆ str        ┆ str      │
│          ┆ str        ┆ str        ┆            ┆        ┆       ┆       ┆            ┆          │
╞══════════╪════════════╪════════════╪════════════╪════════╪═══════╪═══════╪════════════╪══════════╡
│ 54       ┆ null       ┆ null       ┆ null       ┆ null   ┆ null  ┆ null  ┆ Arthropoda ┆ Animalia │
│ 10753682 ┆ null       ┆ null       ┆ Arthropoda ┆ null   ┆ null  ┆ null  ┆ Arthropoda ┆ Animalia │
└──────────┴────────────┴────────────┴────────────┴────────┴───────┴───────┴────────────┴──────────┘





